In [9]:
import numpy as np 
import pandas as pd
from Algorithms import A_C_N, DecisionRuleCreatorFromDecisionTable
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [10]:
DecisionTable = pd.read_csv("./Datasets/car_evaluation.csv")
DecisionTable.columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
DecisionTable

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [11]:
train_df, test_df = train_test_split(DecisionTable, train_size=0.75, stratify=DecisionTable["class"])


In [12]:
train_df_rules = DecisionRuleCreatorFromDecisionTable(train_df)

100%|██████████████████████████████████████| 1295/1295 [00:02<00:00, 516.08it/s]


In [13]:
model = A_C_N(C="EAR", N="cover")

In [14]:
correct = 0
no_result = 0

most_common_train = train_df_rules['class'].mode()[0]

for i in tqdm(range(len(test_df))):
    res = model.solve(train_df_rules, test_df.iloc[i][:-1])
    if res is not None:
        correct += int(res['class'].mode()[0] == test_df.iloc[0]['class'])
    else:
        no_result += 1
        correct += int(most_common_train == test_df.iloc[0]['class'])
        

100%|████████████████████████████████████████| 432/432 [00:04<00:00, 101.24it/s]


In [15]:
print(f"Accuracy = {correct / len(test_df) * 100} %")

Accuracy = 72.45370370370371 %


In [16]:
print(f"No result number = {no_result}")

No result number = 6


In [ ]:
S = pd.DataFrame(
[[np.nan,1,1,np.nan,1],
[0,1,0,np.nan,2],
[0,0,0,np.nan,2],
[0,0,0,np.nan,3],
[0,1,np.nan,np.nan,3],
[np.nan,np.nan,np.nan,1,3]],
columns=['f1','f2','f3','f4','class']
)
S

In [ ]:
delta = pd.DataFrame(
[[0,1,0,1]],
columns=['f1','f2','f3','f4']
)
delta = delta.loc[0]
delta

In [ ]:
ALG = A_C_N(C="AR", N="cover")
ALG.solve(S, delta=delta)

In [2]:
round(478000.25391578674/10000, 2)

47.8

In [ ]:
def R_SR(S):
    """
    input: S - system of decision rules (pandas DataFrame)
    output: subset of S which has reduced by SR reduction (pandas DataFrame)
    """
    to_remove = set()
    n = len(S)

    for i in range(n):
        for j in range(i+1, n):
            if i in to_remove or j in to_remove:
                continue
            r1, r2 = S.iloc[i][:-1], S.iloc[j][:-1] # Exclude the last column
            # Check if r1 is a subset of r2
            if all(np.isnan(r1[k]) or r1[k] == r2[k] for k in S.columns[:-1]):
                to_remove.add(j)
            # Check if r2 is a subset of r1
            elif all(np.isnan(r2[k]) or r2[k] == r1[k] for k in S.columns[:-1]):
                to_remove.add(i)

    return S.drop(S.index[list(to_remove)])


def R_AD(S):
    """
    input: S - system of decision rules (pandas DataFrame)
    output: subset of S which has reduced by AD reduction (pandas DataFrame)
    """
    to_remove = set()
    n = len(S)

    for i in range(n):
        for j in range(i+1, n):
            if i in to_remove or j in to_remove:
                continue
            r1, r2 = S.iloc[i], S.iloc[j]
            # Check if r1 is a subset of r2
            if all(np.isnan(r1[k]) or r1[k] == r2[k] for k in S.columns):
                to_remove.add(j)
            # Check if r2 is a subset of r1
            elif all(np.isnan(r2[k]) or r2[k] == r1[k] for k in S.columns):
                to_remove.add(i)

    return S.drop(S.index[list(to_remove)])


def SAlphaStep(S, alpha):
    """
    input: S - system of decision rules (pandas DataFrame)
           alpha - s tuple of the form (a_i = delta_j)
    output: S_alpha - subset of S as defined in the paper.(just for 1 attribute) (pandas DataFrame)
    """
    
    attr, value = alpha

    # Keep rows where the attr is NaN or equals the specified value
    S = S[(S[attr].isna()) | (S[attr] == value)]
    
    #Make NaN the values
    S.loc[~S[attr].isna(), attr] = np.nan

    return S


def SPlus(S):
    """
    input: S - system of decision rules (pandas DataFrame)
    output: S_plus - subset of S as defined in the paper. (pandas DataFrame)
    """
    non_nan_counts = S.notna().sum(axis=1)

    max_non_nan = non_nan_counts.max()

    S_plus = S[non_nan_counts == max_non_nan]
    
    return S_plus


def SMax(S_plus):
    """
    input: S_plus - system of decision rules with length d (pandas DataFrame)
    output: S_max - subset of S as defined in the paper. (pandas DataFrame)
    """

    columns_to_check = S_plus.columns[:-1]

    S_max = S_plus.drop_duplicates(subset=columns_to_check, keep='first')
    
    return S_max


def NCover(S_plus):
    """ 
    input: S_plus - system of decision rules with length d (pandas DataFrame)
    output: Node cover of S_plus, set of columns that covers all rows
    """
    B = set()

    while not S_plus.iloc[:, :-1].isna().all().all():
        # Select the first row
        r1 = S_plus.iloc[0][:-1]
        # Create A_r1
        A_r1 = set(r1[r1.notna()].index)
        # Add A_r1 to B
        B.update(A_r1)
        
        # Drop rows that have common elements in their index set with A_r1
        rows_to_drop = []
        for index, row in S_plus.iterrows():
            row = row[:-1]
            A_r = set(row[row.notna()].index)
            if A_r1 & A_r:  # Check for common elements
                rows_to_drop.append(index)
        
        S_plus = S_plus.drop(rows_to_drop)

    return B


def NGreedy(S_plus):
    """
    input: S_plus - subset of S as defined in the paper. (pandas DataFrame)
    output: Node cover of S_plus, set of indecies of columns that covers all rows
    """
    S_max = SMax(S_plus)
    
    B = set()
    uncovered_rows = set(S_max.index)

    while uncovered_rows:
        # Find the column that covers the maximum number of uncovered rows
        max_cover = 0
        max_col = None

        for col in S_max.columns[:-1]: # Exclude the last column
            if col in B:
                continue  # Skip columns that have already been chosen
            cover = S_max.index[S_max[col].notna()].intersection(uncovered_rows)
            if len(cover) > max_cover:
                max_cover = len(cover)
                max_col = col

        if max_col is None:
            # No more columns to cover rows, break out
            break

        # Add the column to B and remove covered rows from consideration
        B.add(max_col)
        uncovered_rows -= set(S_max.index[S_max[max_col].notna()])

    return B


In [ ]:
class A_C_N:
    def __init__(self, C="EAR", N="cover"):
        """
        C - type of problem:
            "AR" - All Rules
            "EAR" - Extended All Rules
            "SR" - Some Rules
            "ESR" - Extended Some Rules
            "AD" - All Decisions
            "EAD" - Extended All Decisions
        N - type of Node Cover Algorithm:
            "cover"
            "greedy"
        """
        self.C = C
        self.N = N
        
    def solve(self, S, delta):
        """
        S - System of Decision Rules (pandas DataFrame)
        delta - tuple of attribute values from the set V_C (a row of a pandas df, without any NaN values)
        """
        
        # AR - All Rules problem, EAR - Extended All Rules problem
        if self.C in ["AR", "EAR"]:
            if self.N == "cover": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                       
                    # Step 2
                    else:
                        Q_plus = SPlus(Q)
                        B = NCover(Q_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            Q = SAlphaStep(Q, alpha)
                            
            elif self.N == "greedy": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                    # Step 2
                    else:
                        Q_plus = SPlus(Q)
                        B = NGreedy(Q_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            Q = SAlphaStep(Q, alpha)
            
            else:
                raise ValueError("N must be 'cover' or 'greedy'.")
            
                
        # SR - Some Rules problem, ESR - Extended Some Rules problem
        elif self.C in ["SR", "ESR"]:
            if self.N == "cover": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                       
                    # Step 2
                    else:
                        P = R_SR(Q)
                        P_plus = SPlus(P)
                        B = NCover(P_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            P = SAlphaStep(P, alpha)
                        Q = P
                            
            elif self.N == "greedy": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                       
                    # Step 2
                    else:
                        P = R_SR(Q)
                        P_plus = SPlus(P)
                        B = NGreedy(P_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            P = SAlphaStep(P, alpha)
                        Q = P
            
            else:
                raise ValueError("N must be 'cover' or 'greedy'.")
                
        # AD - All Decisions problem, EAD - Extended All Decisions problem
        elif self.C in ["AD", "EAD"]:
            if self.N == "cover": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                       
                    # Step 2
                    else:
                        P = R_AD(Q)
                        P_plus = SPlus(P)
                        B = NCover(P_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            P = SAlphaStep(P, alpha)
                        Q = P
                            
            elif self.N == "greedy": # with "cover" NodeCover method
                Q = S.copy()
                while True:
                    # Step 1
                    if (Q.empty or Q.iloc[:, :-1].isna().all().all()):
                        if Q.empty:
                            print("There is no such rule")
                            return
                        else:
                            row_indecies = Q.index.tolist()
                            return S.loc[row_indecies]
                       
                    # Step 2
                    else:
                        P = R_AD(Q)
                        P_plus = SPlus(P)
                        B = NGreedy(P_plus)
                        for attr in B:
                            alpha = (attr, delta[attr])
                            P = SAlphaStep(P, alpha)
                        Q = P
            
            else:
                raise ValueError("N must be 'cover' or 'greedy'.")
                
        # Wrong problem type      
        else: 
            raise ValueError("C must be one of {'AR', 'EAR', 'SR', 'ESR', 'AD', 'EAD'}")

In [ ]:
S = pd.DataFrame(
[[np.nan,1,1,np.nan,1],
[0,1,0,np.nan,2],
[0,0,0,np.nan,2],
[0,0,0,np.nan,3],
[0,1,np.nan,np.nan,3],
[np.nan,np.nan,np.nan,1,3]],
columns=['f1','f2','f3','f4','class']
)
S

In [ ]:
delta = pd.DataFrame(
[[0,1,0,2]],
columns=['f1','f2','f3','f4']
)
delta = delta.loc[0]
delta

In [ ]:
A_SR_N = A_C_N(C="AD", N="greedy")
A_SR_N.solve(S, delta=delta)

In [ ]:
S = pd.DataFrame(
[[np.nan,1,1,np.nan,1],
[0,1,0,np.nan,2],
[0,np.nan,0,np.nan,2],
[0,0,0,np.nan,3],
[0,1,np.nan,np.nan,3],
[np.nan,np.nan,np.nan,np.nan,3]],
columns=['f1','f2','f3','f4','class']
)
S

In [ ]:
def R_AD(S):
    to_remove = set()
    n = len(S)

    for i in range(n):
        for j in range(i+1, n):
            if i in to_remove or j in to_remove:
                continue
            r1, r2 = S.iloc[i], S.iloc[j]
            # Check if r1 is a subset of r2
            if all(np.isnan(r1[k]) or r1[k] == r2[k] for k in S.columns):
                to_remove.add(j)
            # Check if r2 is a subset of r1
            elif all(np.isnan(r2[k]) or r2[k] == r1[k] for k in S.columns):
                to_remove.add(i)

    return S.drop(S.index[list(to_remove)])


In [ ]:
R_AD(S)

In [ ]:
def R_SR(S):
    """
    Reduces S, with SR reduction
    input: S - system of decision rules (pandas DataFrame)
    output: SR_reduced - subset of S as defined in the paper (pandas DataFrame)
    """
    
    attr, value = alpha

    # Keep rows where the attr is NaN or equals the specified value
    S = S[(S[attr].isna()) | (S[attr] == value)]
    
    #Make NaN the values
    S.loc[~S[attr].isna(), attr] = np.nan

    return S

In [ ]:
S1 = SAlphaStep(S, ("f1", 0))
S1

In [ ]:
S2 = SAlphaStep(S1, ("f2", 1))
S2

In [ ]:
S3 = SAlphaStep(S2, ("f3", 1))
S3

In [ ]:
S4 = SAlphaStep(S3, ("f4", 1))
S4

In [ ]:
S4.loc[5]

In [ ]:
S4.index.tolist()

In [ ]:
S4.loc[[0,5]]

In [ ]:
r = S4.loc[5]
r

In [ ]:
r["class"]

In [ ]:
A_EAR_N = A_C_N(C="EAR", N="cover")
A_EAR_N.solve(S, delta=123)

In [ ]:
delta["f1"]

In [ ]:
S = pd.DataFrame(
[[np.nan,np.nan,np.nan,np.nan,1],
[np.nan,np.nan,np.nan,np.nan,2],
[np.nan,np.nan,np.nan,np.nan,2],
[np.nan,np.nan,np.nan,np.nan,3],
[np.nan,np.nan,np.nan,np.nan,3],
[np.nan,np.nan,np.nan,np.nan,3]],
columns=['f1','f2','f3','f4','class']
)
S

In [ ]:
def NGreedy(S_max):
    """
    input: S_max - subset of S as defined in the paper. (pandas DataFrame)
    output: Node cover of S_max, set of indecies of columns that covers all rows
    """
    B = set()
    uncovered_rows = set(S_max.index)

    # Number of columns excluding the last one
    num_columns = S_max.shape[1] - 1

    while uncovered_rows:
        max_cover = 0
        max_col = None

        for col_index in range(num_columns):
            if col_index in B:
                continue  # Skip columns that have already been chosen
                
            cover = S_max.index[S_max[col].notna()].intersection(uncovered_rows)
            if len(cover) > max_cover:
                max_cover = len(cover)
                max_col_index = col_index

        if max_col_index is None:
            # No more columns to cover rows, break out
            break

        # Add the column index to B and remove covered rows from consideration
        B.add(max_col_index)
        uncovered_rows -= set(S_max.index[S_max[max_col].notna()])

    return B

NGreedy(S)

In [ ]:
def NGreedy(S_max):
    """
    input: S_max - subset of S as defined in the paper. (pandas DataFrame)
    output: Node cover of S_max, set of indecies of columns that covers all rows
    """
    B = set()
    uncovered_rows = set(S_max.index)

    while uncovered_rows:
        # Find the column that covers the maximum number of uncovered rows
        max_cover = 0
        max_col = None

        for col in S_max.columns:
            if col in B:
                continue  # Skip columns that have already been chosen
            cover = S_max.index[S_max[col].notna()].intersection(uncovered_rows)
            if len(cover) > max_cover:
                max_cover = len(cover)
                max_col = col

        if max_col is None:
            # No more columns to cover rows, break out
            break

        # Add the column to B and remove covered rows from consideration
        B.add(max_col)
        uncovered_rows -= set(S_max.index[S_max[max_col].notna()])

    return B

NGreedy(S)

In [ ]:
r1 = S.iloc[0][:-1]

In [ ]:
A_r1 = set(r1[r1.notna()].index)
A_r1

In [ ]:
def NGreedy(S_max):
    """
    input: S_max - subset of S as defined in the paper. (pandas DataFrame)
    output: Node cover of S_max, set of indecies of columns that covers all rows
    """
    B = set()
    uncovered_rows = set(S_max.index)

    # Number of columns excluding the last one
    num_columns = S_max.shape[1] - 1

    while uncovered_rows:
        max_cover = 0
        max_col_index = None

        for col_index in range(num_columns):
            if col_index in B:
                continue  # Skip columns that have already been chosen

            cover = S_max.index[S_max.iloc[:, col_index].notna()].intersection(uncovered_rows)
            if len(cover) > max_cover:
                max_cover = len(cover)
                max_col_index = col_index

        if max_col_index is None:
            # No more columns to cover rows, break out
            break

        # Add the column index to B and remove covered rows from consideration
        B.add(max_col_index)
        uncovered_rows -= set(S_max.index[S_max.iloc[:, max_col_index].notna()])

    return B

In [ ]:
class A_C_N:
    def __init__(self, C="EAR", N="cover"):
        """
        C - type of problem:
            AR - All Rules
            EAR - Extended All Rules
            SR - Some Rules
            ESR - Extended Some Rules
            AD - All Decisions
            EAD - Extended All Decisions
        N - type of Node Cover Algorithm:
            cover
            greedy
        """
        self.C = C
        self.N = N
        
    def forward(self, S, delta):
        """
        S - System of Decision Rules (pandas DataFrame)
        delta - tuple of attribute values from the set V_C
        """
        if self.C == "EAR" and self.N == "cover":
            Q = S.copy()
            
            Q_plus = SPlus(Q)
            B = NCover(Q_plus)
            
            
            

In [ ]:
def SAlpha(S, alpha):
    """
    input: S - system of decision rules (pandas DataFrame)
           alpha - tuple of equations of the for ((a_i = delta_j), ...)
    output: S_alpha - subset of S as defined in the paper. (pandas DataFrame)
    """

    for col, value in alpha:
        # Keep rows where the column is NaN or equals the specified value
        S = S[(S[col].isna()) | (S[col] == value)]

    for col, _ in alpha:
        S.loc[~S[col].isna(), col] = np.nan

    return S

In [ ]:
SAlpha(S_max, (("f1", 0), ("f4", 3), ("f2", 1)))

In [ ]:
Ncover(S_plus_ex)

In [ ]:
r1 = S_plus_ex.iloc[0]
# Create A_r1: set of indices (integer positions) where r1 is not NaN
A_r1 = set(r1[r1.notna()].index.tolist())
A_r1

In [ ]:
def Ncover(S_plus):
    B = set()
    while len(S_plus):
        r1 = S_plus.iloc[0]
        A_r1 = set(np.where(r1[:-1].notna())[0])
        print(A_r1)
        B = B.union(A_r1)
        for row in range(len(S_plus)):
            drop 
            r = S_plus.iloc[row]
            A_r = set(np.where(r[:-1].notna())[0])
            print(A_r)
            if A_r1 & A_r:
                S_plus = S_plus.drop(row)
        print(S_plus)
        break
    return B
                
            

In [ ]:
Ncover(S_plus_ex)

In [ ]:
import pandas as pd

def process_dataframe(df):
    B = set()

    while not df.empty:
        # Select the first row
        r1 = df.iloc[0]
        # Create A_r1: set of indices (integer positions) where r1 is not NaN
        A_r1 = set(r1[r1.notna()].index.tolist())
        # Add A_r1 to B
        B.update(A_r1)
        
        # Drop rows that have common elements in their index set with A_r1
        rows_to_drop = []
        for index, row in df.iterrows():
            A_r = set(row[row.notna()].index.tolist())
            if A_r1 & A_r:  # Check for common elements
                rows_to_drop.append(index)
        
        df = df.drop(rows_to_drop)

    return B


In [ ]:
for row in S_plus_ex:
    print(row)

In [ ]:
r1 = S_plus_ex.iloc[0]
r1

In [ ]:
len(r1)

In [ ]:
len(S_plus)

In [ ]:
B = set()
r1 = S_plus_ex.iloc[0]
B = B.union([i for i in range(len(r1)) if r1[i] != np.nan])

In [ ]:
B

In [ ]:
if r1[0]!=np.nan:
    print("dff")

In [ ]:
r1[r1.notna()].index

In [ ]:
set(np.where(r1.notna())[0])

In [ ]:
np.where(r1.notna())

In [ ]:
non_nan_indices = set(np.where(row[:-1].notna())[0])

In [ ]:
set(np.where(r1[:-1].notna())[0])

In [ ]:
DecisionRules = pd.read_csv("./Datasets/car_DRules.csv")

In [ ]:
DecisionRules

In [ ]:
DecisionRules.iloc[579]

In [ ]:
S_plus = Splus(DecisionRules)
S_plus

In [ ]:
S_plus.loc[579][0]

In [ ]:
S_plus.iloc[0][0]

In [ ]:
S_max = Smax(S_plus)
S_max

In [ ]:
example_DRules = pd.read_csv("./Datasets/example_DRules.csv")
example_DRules

In [ ]:
S_plus = Splus(example_DRules)
S_plus

In [ ]:
example_DTable = pd.DataFrame(
[[1,1,1,1],
[0,1,0,2],
[0,1,0,2],
[0,0,1,3],
[0,0,1,3]],
columns=['f1','f2','f3','class']
)

In [ ]:
example_DTable

In [ ]:
S_plus = Splus(example_DTable)
S_plus

In [ ]:
S_max = Smax(S_plus)
S_max